In [1]:
using BenchmarkTools

In [2]:
W = trues(30000, 30001)
W[rand(1:length(W), round(Int, √length(W)))] .= 0
W

30000×30001 BitMatrix:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1   

In [3]:
function get_S(W)
    zeroidxs = getindex.(findall(iszero, W), [1 2])
    S1 = Set(zeroidxs[:,1])
    S2 = Set(zeroidxs[:,2])
    return S1, S2
end

get_S (generic function with 1 method)

In [4]:
@btime get_S($W)

  1.402 s (37 allocations: 3.04 MiB)


(Set([11950, 7685, 3406, 28804, 27640, 28576, 1090, 2015, 17088, 11280  …  4724, 25754, 21664, 10808, 8992, 8546, 24824, 4159, 19700, 4496]), Set([29965, 1703, 12427, 3406, 28804, 28900, 23970, 27640, 28576, 2015  …  1711, 15271, 4724, 4496, 19700, 21664, 4159, 8992, 24824, 25754]))

In [5]:
function get_S1(W)
    zeroidxs = getindex.(findall(iszero, W), [1 2])
    S1 = unique(zeroidxs[:,1])
    S2 = unique(zeroidxs[:,2])
    return S1, S2
end

get_S1 (generic function with 1 method)

In [6]:
@btime get_S1($W)

  1.395 s (107 allocations: 4.42 MiB)


([2004, 17474, 9903, 24921, 27646, 10011, 2616, 5704, 24106, 25498  …  21396, 5326, 4142, 29187, 1734, 18026, 4938, 22325, 21462, 25089], [1, 3, 4, 6, 7, 8, 11, 14, 15, 16  …  29983, 29985, 29986, 29988, 29990, 29996, 29998, 29999, 30000, 30001])

In [7]:
sort.(unique.(collect.(get_S(W)))) == sort.(get_S1(W))

true

In [8]:
function get_S2(W)
    m, n = size(W)
    b1, b2 = fill(false, m), fill(false, n)
    Threads.@threads for j in 1:n
        for i in 1:m
            @inbounds if W[i, j] == 0
                b1[i] = b2[j] = true
            end
        end
    end
    b1, b2
end

get_S2 (generic function with 1 method)

In [9]:
@btime get_S2($W)

  74.266 ms (65 allocations: 64.98 KiB)


(Bool[0, 1, 1, 0, 0, 1, 1, 0, 1, 1  …  1, 1, 1, 1, 1, 0, 0, 1, 1, 1], Bool[1, 0, 1, 1, 0, 1, 1, 1, 0, 0  …  0, 0, 0, 0, 1, 0, 1, 1, 1, 1])

In [10]:
sort.(get_S1(W)) == findall.(get_S2(W))

true

In [11]:
get_S(W) |> typeof

Tuple{Set{Int64}, Set{Int64}}

In [12]:
get_S1(W) |> typeof

Tuple{Vector{Int64}, Vector{Int64}}

In [13]:
get_S2(W) |> typeof

Tuple{Vector{Bool}, Vector{Bool}}